# Previsão Idade PCA

Neste notebook será abordado o problema de estimação de idade, utilizando agora a técnica de PCA. Tal como nos notebooks passados, é preciso, numa fase inicial, carregar os dados dos ficheiros previamente criados. De seguida será feita a separação dos dados para treino/teste (com a respetiva passagem de pixeis para as componentes principais, reduzindo significativamente a dimensão do input) e por fim testar-se-á o sucesso da rede. Para tal usaremos não só a métrica da accuracy como de 1Off (qual a percentagem de acertar no bin correto ou num diretamente adjacente a este, +/-1).

### Imports necessários

In [1]:
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import os, shutil
import matplotlib.pyplot as plt
from PIL import Image



## Recuperação dos dados dos ficheiros relativamente às imagens/labels

Leitura do ficheiro csv correspondente a todas as labels e anexação destas à lista results

In [2]:
#Leitura do ficheiro csv correspondente a todas as labels
#anexação destas a lista results
import csv
results = []
with open("HalfData.csv") as csvfile:
    reader = csv.reader(csvfile, quoting=csv.QUOTE_ALL) # change contents to floats
    for row in reader: # each row is a list
        results.append(row)

Processamento das idades e conversão destas para os respetivos bins.
Tal como referido anteriormente estes bins seguem a distribuição normal dos dados do dataset.

In [3]:
#Processamento das idades e conversão para lista
labelslist = []
label =""
for i in range(len(results)):
    age = int(results[i][0])
    if(age>=18 and age<=21):
        label = "18_21"
    if(age>=22 and age<=24):
        label = "22_24"
    if(age>=25 and age<=28):
        label = "25_28"
    if(age>=29 and age<=33):
        label = "29_33"
    if(age>=34 and age<=40):
        label = "34_40"
    if(age>=41 and age<=47):
        label = "41_47"
    if(age>=48 and age<=55):
        label = "48_55"
    if(age>=56 and age<=65):
        label = "56_65"
    labelslist.append(label)

In [4]:
labels = np.reshape(labelslist,len(labelslist))
labels

array(['56_65', '22_24', '34_40', ..., '25_28', '48_55', '25_28'],
      dtype='<U5')

Leitura do ficheiro relativo aos pontos das fotos.
Neste caso foi utilizado o que possuía metade da informação do dataset (13877 fotos).
Este foi guardado num ficheiro binário por uma questão de redução de dimensão (uma vez que este contém 13877 * 256 * 256 floats)

In [5]:
teste = np.reshape(np.fromfile("HalfData"),(13877,256,256))

## Divisão dos dados para treino e teste


Uma vez que é necessária uma divisão do dataset para treino e teste, utilizamos o train_test_split com a flag de stratify (garantindo que a distribuição dos dados se mantem nas versões "reduzidas") e com random_state, permitindo assim que a operação se torne determinística (os mesmos dados irão ser divididos sempre para os mesmos conjuntos de treino e teste, enquanto que o valor desta flag se mantenha constante)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(teste,labels, test_size=0.20, stratify=labels, random_state = 2)

Tendo os dados separados para treino e teste, podemos aplicar a técnica de PCA. Deste modo, obtemos as principais componentes dos dados e serão estas passadas à rede.


Decidimos incluir as primeiras 10 componentes, que possuem 63% dos dados (neste momento cada componente adiciona pouco mais de 1%...).
Começamos por faze-lo para as de treino com fit_transform e de seguida, para teste, com transform apenas fazendo com que esta siga as mesmas propriedades que a de treino.

In [76]:
#PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
principalComponents = pca.fit_transform(np.reshape(X_train,(len(X_train),256*256)))
principalDf = pd.DataFrame(data = principalComponents)
             #, columns = ['principal component 1', 'principal component 2'])

In [8]:
#Peso de cada uma das 10 componentes
pesos=pca.explained_variance_ratio_
pesos

array([0.30319141, 0.09155268, 0.05868098, 0.0468638 , 0.0333811 ,
       0.02709735, 0.02222755, 0.01742475, 0.01732365, 0.01343905])

In [9]:
#Somatório do peso das primeiras 10 componentes
info=np.cumsum(pesos)
info[9]

0.631182332008199

In [10]:
#PCA para os dados de teste
from sklearn.decomposition import PCA
principalComponents2 = pca.transform(np.reshape(X_test,(len(X_test),256*256)))
principalDf2 = pd.DataFrame(data = principalComponents)



Construção das labels para a rede (aplicação de one-hot-encoding)

In [11]:
train_labels = pd.get_dummies(y_train)
train_labels = train_labels.to_numpy()
test_labels = pd.get_dummies(y_test)
test_labels = test_labels.to_numpy()

## Criação e treino da rede -> PCA 6layers

Uma vez que os dados a passar a rede são as principal components das fotos, consideramos que esta rede tinha de ser composta por múltiplas camadas Dense, de modo a maximizar a aprendizagem da mesma.

Esta possui uma camada inicial com 10 neurnios (dimensão do input -> 10 principal components) e uma final com 8 (número de classes existentes -> dimensão do output). De resto utiliza camadas variadas entre 128,256 e 512.


In [63]:
from tensorflow.keras import models, layers
network = models.Sequential()
network.add(layers.Dense(10, activation='relu', input_shape=(10,)))
network.add(layers.Dense(128, activation='relu'))
network.add(layers.Dense(256, activation='relu'))
network.add(layers.Dense(512, activation='relu'))
network.add(layers.Dense(128, activation='relu'))
network.add(layers.Dense(8, activation='softmax'))

In [64]:
network.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
network.fit(principalComponents, train_labels, epochs=20,batch_size=16)


Epoch 1/20
694/694 [==============================] - 2s 2ms/step - loss: 2.0846 - accuracy: 0.1693
Epoch 2/20
694/694 [==============================] - 2s 2ms/step - loss: 2.0375 - accuracy: 0.1786
Epoch 3/20
694/694 [==============================] - 2s 2ms/step - loss: 2.0288 - accuracy: 0.1805
Epoch 4/20
694/694 [==============================] - 2s 2ms/step - loss: 2.0235 - accuracy: 0.1864
Epoch 5/20
694/694 [==============================] - 2s 2ms/step - loss: 2.0198 - accuracy: 0.1870
Epoch 6/20
694/694 [==============================] - 2s 2ms/step - loss: 2.0153 - accuracy: 0.1903
Epoch 7/20
694/694 [==============================] - 2s 2ms/step - loss: 2.0154 - accuracy: 0.1907
Epoch 8/20
694/694 [==============================] - 2s 2ms/step - loss: 2.0107 - accuracy: 0.1886
Epoch 9/20
694/694 [==============================] - 2s 3ms/step - loss: 2.0090 - accuracy: 0.1896
Epoch 10/20
694/694 [==============================] - 2s 2ms/step - loss: 2.0050 - accuracy: 0.1978

Após o treino da rede, podemos passar à análise do sucesso da mesma. Assim, verificaremos a sua accuracy para fase de teste e posterior valor para 1Off, 1OffByClass e AccuracybyClass. Estas últimas métricas permitiram-nos perceber o sucesso do insucesso da rede, verificando assim se quando esta erra, foi por achar que seria uma idade próxima(bin adjacente) ou se é um palpite "completamente" aleatório.

Como podemos ver pelo valor abaixo, a accuracy em teste é bastante proxima à de treino, permitindo assim concluir que a rede efetivamente aprendeu (por menos que tenha sido) e não entrou em OF.

In [77]:
test_loss, test_acc = network.evaluate(principalComponents2, test_labels)

87/87 [==============================] - 0s 2ms/step - loss: 2.0401 - accuracy: 0.1913


In [83]:
predictionsBOF = network.predict(principalComponents2)

### One Off

Como podemos verificar abaixo, esta rede apesar de possuir uma baixíssima capacidade de estimar a idade (19%), acerta quase metade das vezes (49%) quando incluindo o bin adjacente (ou seja, esta erra mais por não conseguir aprender especificamente os parâmetros para cada bin, mas efetivamente aprende a distinguir entre eles)

In [84]:

def oneOff(predictions,correct):
    acc = 0
    for i in range(len(predictions)):
        if(np.argmax(predictions[i])==np.argmax(correct[i])+1):
            acc = acc + 1 ;
        if(np.argmax(predictions[i])==np.argmax(correct[i])-1): 
            acc = acc + 1 ;
        if(np.argmax(predictions[i])==np.argmax(correct[i])):
            acc = acc + 1 ;
    return (acc/len(predictions))*100;

In [85]:
oneOff(predictionsBOF,test_labels)

49.92795389048991

### One Off By Class

Uma vez que o valor de One off foi tão "bom" comparando à accuracy geral, consideramos que seria relevanta a sua análise por bin. Aqui surge um pormenor interessante, que é o valor "altíssimo" de precisão nos bins de 22 a 33 (maior concentração dos dados). Assim consideramos relevante, de modo a perceber melhor o porque desta especificidade, implementar também a accuracy por bin.

In [88]:
def oneOfbyClass(predictions,correct):
    acc = [0,0,0,0,0,0,0,0]
    listaLabels = ["18_21","22_24","25_28","29_33","34_40","41_47","48_55","56_65"]
        
    for i in range(len(predictions)):
        if(np.argmax(predictions[i])==np.argmax(correct[i])+1):
            acc[np.argmax(correct[i])] = acc[np.argmax(correct[i])] + 1 ;
        if(np.argmax(predictions[i])==np.argmax(correct[i])-1): 
            acc[np.argmax(correct[i])] = acc[np.argmax(correct[i])] + 1 ;
        if(np.argmax(predictions[i])==np.argmax(correct[i])):
            acc[np.argmax(correct[i])] = acc[np.argmax(correct[i])] + 1 ;
    
    listaIndices = [0,0,0,0,0,0,0,0]
    for i in range(len(correct)):
        listaIndices[np.argmax(correct[i])] = listaIndices[np.argmax(correct[i])] + 1
    
    newlista = []
    for i in range(len(listaIndices)):
        newlista.append(((acc[i]/listaIndices[i])*100,listaLabels[i]))
        
    
    
    return newlista;

In [89]:
oneOfbyClass(predictionsBOF,test_labels)

[(2.05761316872428, '18_21'),
 (79.1044776119403, '22_24'),
 (83.08351177730194, '25_28'),
 (91.1832946635731, '29_33'),
 (18.341708542713565, '34_40'),
 (29.283489096573206, '41_47'),
 (24.679487179487182, '48_55'),
 (33.82899628252788, '56_65')]

### Accuracy By Class

Tal como podemos ver pelos valores abaixo, o motivo de valores tão grandes de accuracy nesses bins é devido ao facto de uma enorme precisão para o bin dos 25_28 (bin mais denso). Deste modo concluímos que a rede, quando não é capaz de identificar características específicas de um modelo, adivinha o que constitui o bin mais denso (mais provável de acertar), o que por uma questão de 1Off torna os bins adjacentes bastantes precisos nesta métrica, quando em accuracy absoluta são bastante "fracos" (1.7% e 0.23%).

In [90]:
def accByClass(predictions,correct):
    acc = [0,0,0,0,0,0,0,0]
    listaLabels = ["18_21","22_24","25_28","29_33","34_40","41_47","48_55","56_65"]

    for i in range(len(predictions)):
        if(np.argmax(predictions[i])==np.argmax(correct[i])):
            acc[np.argmax(correct[i])] = acc[np.argmax(correct[i])] + 1 ;
    
    listaIndices = [0,0,0,0,0,0,0,0]
    for i in range(len(correct)):
        listaIndices[np.argmax(correct[i])] = listaIndices[np.argmax(correct[i])] + 1
    
    newlista = []
    for i in range(len(listaIndices)):
        newlista.append(((acc[i]/listaIndices[i])*100,listaLabels[i]))
    
    return newlista;

In [91]:
accByClass(predictionsBOF,test_labels)

[(0.411522633744856, '18_21'),
 (1.791044776119403, '22_24'),
 (80.08565310492506, '25_28'),
 (0.23201856148491878, '29_33'),
 (15.326633165829145, '34_40'),
 (3.115264797507788, '41_47'),
 (15.064102564102564, '48_55'),
 (11.524163568773234, '56_65')]